In [18]:
from bs4 import BeautifulSoup
import random
import time
import undetected_chromedriver as uc
import pandas as pd
import requests
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

In [19]:
user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; rv:101.0) Gecko/20100101 Firefox/101.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Version/14.0.3 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36 Edge/96.0.1054.62",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36 OPR/80.0.4170.72",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; Trident/7.0; ASL 1.0; .NET4.0E; .NET4.0C; InfoPath.3) like Gecko",
            "Mozilla/5.0 (Linux; Android 10; Pixel 4 XL Build/QD1A.190805.001) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.115 Mobile Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
            "Mozilla/5.0 (Android 10; Mobile; rv:85.0) Gecko/85.0 Firefox/85.0",
            "Opera/9.80 (Windows NT 6.1; U; en) Presto/2.12.388 Version/12.17",
            "Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/14.0 Chrome/73.0.3683.90 Mobile Safari/537.36",
            "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",
            "Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)"
        ]

In [20]:
rendom_agent = random.choice(user_agents)
print(rendom_agent)

url = "https://fbref.com/en/comps/12/La-Liga-Stats"

option = uc.ChromeOptions()
option.add_argument(f"user-agent={rendom_agent}")
option.add_argument("--no-sandbox")
option.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(options=option)


driver.get(url)
time.sleep(random.uniform(2, 3))
print(driver.current_url)
time.sleep(random.uniform(2, 3))
driver.maximize_window()

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Version/14.0.3 Safari/537.36
https://fbref.com/en/comps/12/La-Liga-Stats


In [21]:
soup = BeautifulSoup(driver.page_source, "lxml")
previous = soup.find('div', class_='prevnext')
prev = previous.text.strip()
prev[:prev.find('\n')]

'Previous Seaso'

In [22]:
headers = []
teams_data = []

while True:
    soup = BeautifulSoup(driver.page_source, "lxml")
    class1 = soup.find_all("table", class_="stats_table sortable min_width force_mobilize now_sortable")
    table = class1[0]
    previous = soup.find('div', class_='prevnext')
    prev = previous.text.strip()

    # Try to click the close-ad button if it appears
    try:
        ad_button = driver.find_element(By.XPATH, '//*[@id="modal-close"]')
        ad_button.click()
        time.sleep(random.uniform(1.5, 2.5))
    except ElementNotInteractableException as e:
        pass  # No ad to close

    # Scrape headers (only once ideally)
    if not headers:
        for i in table.find_all("thead"):
            for j in i.find_all("tr"):
                for k in j.find_all("th"):
                    headers.append(k.text.strip())

    # Scrape data
    for i in table.find_all("tbody"):
        for j in i.find_all("tr"):
            data = []
            for k in j.find_all(["td", "th"]):
                data.append(k.text.strip())
            teams_data.append(data)

    time.sleep(random.uniform(2, 3))

    # Try clicking the next/prev button
    try:
        button = driver.find_element(By.CSS_SELECTOR, '#meta > div:nth-child(2) > div > a')
    except ElementNotInteractableException as e:
        try:
            button = driver.find_element(By.CSS_SELECTOR, '#meta > div:nth-child(2) > div > a.button2.prev')
        except ElementNotInteractableException as e:
            print("No more pages.")
            break  # Exit the loop if no buttons exist
    
    if prev.split('\n', 1)[0] == "Previous Season":
        button.click()
    else:
        break

In [23]:
teams_data

[['1',
  'Barcelona',
  '38',
  '28',
  '4',
  '6',
  '102',
  '39',
  '+63',
  '88',
  '2.32',
  '91.5',
  '41.9',
  '+49.5',
  '+1.30',
  '45,953',
  'Robert Lewandowski - 27',
  'Iñaki Peña',
  '→ Champions League via league finish'],
 ['2',
  'Real Madrid',
  '38',
  '26',
  '6',
  '6',
  '78',
  '38',
  '+40',
  '84',
  '2.21',
  '75.3',
  '42.8',
  '+32.5',
  '+0.86',
  '69,807',
  'Kylian Mbappé - 31',
  'Thibaut Courtois',
  '→ Champions League via league finish'],
 ['3',
  'Atlético Madrid',
  '38',
  '22',
  '10',
  '6',
  '68',
  '30',
  '+38',
  '76',
  '2.00',
  '64.6',
  '33.4',
  '+31.2',
  '+0.82',
  '60,883',
  'Alexander Sørloth - 20',
  'Jan Oblak',
  '→ Champions League via league finish'],
 ['4',
  'Athletic Club',
  '38',
  '19',
  '13',
  '6',
  '54',
  '29',
  '+25',
  '70',
  '1.84',
  '53.0',
  '37.8',
  '+15.2',
  '+0.40',
  '48,420',
  'Oihan Sancet - 15',
  'Unai Simón',
  '→ Champions League via league finish'],
 ['5',
  'Villarreal',
  '38',
  '20',
  '10

In [24]:
data = pd.DataFrame(teams_data, columns=headers[:19])
data

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Barcelona,38,28,4,6,102,39,+63,88,2.32,91.5,41.9,+49.5,+1.30,"45,953",Robert Lewandowski - 27,Iñaki Peña,→ Champions League via league finish
1,2,Real Madrid,38,26,6,6,78,38,+40,84,2.21,75.3,42.8,+32.5,+0.86,"69,807",Kylian Mbappé - 31,Thibaut Courtois,→ Champions League via league finish
2,3,Atlético Madrid,38,22,10,6,68,30,+38,76,2.00,64.6,33.4,+31.2,+0.82,"60,883",Alexander Sørloth - 20,Jan Oblak,→ Champions League via league finish
3,4,Athletic Club,38,19,13,6,54,29,+25,70,1.84,53.0,37.8,+15.2,+0.40,"48,420",Oihan Sancet - 15,Unai Simón,→ Champions League via league finish
4,5,Villarreal,38,20,10,8,71,51,+20,70,1.84,64.8,44.4,+20.4,+0.54,"18,266",Ayoze Pérez - 19,Diego Conde,→ Champions League via league finish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,16,Málaga,38,12,9,17,39,53,-14,33,0.87,,José Francisco Sánchez González - 8,Pedro Luis Jaro,,None,None,None,None
740,17,Espanyol,38,7,16,15,29,44,-15,30,0.79,,Michel Pineda - 5,Thomas N'Kono,→ Spanish 1/2 Relegation/Promotion play-offs,None,None,None,None
741,18,Real Betis,38,9,11,18,36,55,-19,29,0.76,,Hipólito Rincón - 8,Nery Pumpido,→ Spanish 1/2 Relegation/Promotion play-offs,None,None,None,None
742,19,Real Murcia,38,9,6,23,27,58,-31,24,0.63,,José Mejías - 8,Amador Lorenzo,Relegated,None,None,None,None
